In [8]:
import praw
import prawcore
import pandas as pd
import numpy as np 
import glob
import os
import re
import math
import sklearn

from collections import Counter
from itertools import chain, combinations
from more_itertools import pairwise
from tqdm import tqdm
from datetime import datetime, date

import networkx as nx
import seaborn
import matplotlib.pyplot as plt
import en_core_web_sm

# import plotly.graph_objs as go
# import plotly.tools as tls
# import plotly.express as px

from spacy import load
import warnings
warnings.filterwarnings("ignore", message=r"\[W008\]", category=UserWarning)

In [4]:
from transformers import pipeline
sentiment_toxicity = pipeline("sentiment-analysis", model="unitary/toxic-bert")

tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

C:\Users\ray\anaconda3\envs\Pytorch\lib\site-packages\torch\nn\modules\module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [9]:
path = 'data'
user_dir = 'user_data'
reddit = praw.Reddit(
    client_id='4bDjCY6y8ncrc3kLrBbpBg',
    client_secret='zPHZKfk9S666S9IxR5HtvkZ83ufNxw',
    user_agent='webcrawler created for IS596'
)
nlp = en_core_web_sm.load()

In [10]:
def normalize(text):
    norm_text = []

    for token in text:
        if not token.is_punct and not token.is_stop and not token.is_space:
            norm_text.append(token.lemma_.lower())

    norm_text = ' '.join(norm_text)
    norm_text = re.sub(r'(?:^| )\w(?:$| )', ' ', norm_text).strip()  # removes single characters
    norm_text = re.sub(r'[^a-zA-Z0-9 ]', '', norm_text)
    
    return norm_text

In [11]:
def get_user_status(path):
    try:
        user.is_suspended
        print('sus')
        user_status = 'suspended'
    except AttributeError:
        user_status = 'active'
    except:
        user_status = 'deleted'
    return user_status

### For known bots

In [12]:
# infile_name = f'{path}/bot_list.csv'
# infile = open(infile_name, 'r')
# file_df = pd.read_csv(infile)
# user_names = file_df.user_name
# print(user_names)

0         KickOpenTheDoorBot
1                 Canna_Tips
2                MrTechnodad
3                    BigJets
4                    properu
                ...         
2495                 Korpels
2496    LEAPYEAR_FOR_SCIENCE
2497              LadyMoirai
2498    Late-Requirement-653
2499               Lauris024
Name: user_name, Length: 2500, dtype: object


### For suspected bots

In [13]:
botdefense = reddit.subreddit('botdefense').new(limit=500)
user_names = []

for submission in botdefense:
    user_names.append(submission.title[13:])
user_names = user_names[1:]
print(user_names[:10])

['Humzsdmj', 'Breawsfdg', 'Outsidryzujhx', 'Fantastic-Camp-9597', 'Newbornsale480', 'WeepyBingo', 'NervouslyStick', 'blogdesmartnx', 'CalligrapherSimple50', 'Shantawtaylor', 'Daddidounet', 'Appropriatbhfd', 'advisoryArctiid943', 'BriefOrdera', 'Strongjoachim', 'Federicebl', 'Born_Ad9041', 'Anteatxcv', 'veryBeat', 'Negotiwqdcfv', 'Dry_Association1880', 'GentlyPleafsed', 'Spiritual-Solid-5119', 'Activesxgv', 'gonzdarr', 'Daperasf', 'ferotgeio', 'Visiblecgb', 'zpayment', 'Weak_Past_4575', 'Rare_Bee649', 'Compwfxfg', 'Noelicoszv', 'Ok-Lock-3079', 'Ambitiazxc', 'Independencfg', 'OkTry5944', 'Positivdfsgh', 'PieceKitchen9791', 'Animatotgjug', 'Basiqwfdfd', 'Alarmeezuyxu', 'Cablerugckjk', 'WildWorth8147', 'Strange-Debt3043', 'actuallyAnswer599', 'Necessary-Wonder3842', 'Loud_Bid7147', 'Aggravating-Most2548', 'Gloomyiolinist', 'Previoysitug', 'Emptysdadzfgh', 'Catchwsfg', 'Alarminsdfggf', 'ShaggilySeek', 'Superqwsfg', 'Hapucotadd', 'Completrftyaag', 'Workinfhykxt', 'Any-Gap-2903', 'Personasdfg

In [ ]:
rows = []
for user_name in tqdm(user_names):
    try:
        user_dict = {}
        user = reddit.redditor(user_name)
        user_dict['username'] = user_name
        user = reddit.redditor(user_name)

        if get_user_status(user) == 'active' and str(user_name) not in 'nan': # checks that user isn't suspended/deleted
            if not user.is_mod: #ignore mods

                comment_array = []
                timestamps = []
                reply_timestamps = []
                comment_toxicities = []

                comment_karma = user.comment_karma
                submission_karma = user.link_karma

                karma_ratio = round(comment_karma/submission_karma, 4)

                try:
                    for this_comment in user.comments.new(limit=15):

                        parent_comment_id = this_comment.parent_id
                        if parent_comment_id.startswith('t3'):
                            parent_comment_id = parent_comment_id[3:]
                            parent = reddit.submission(parent_comment_id)
                        else:
                            parent = reddit.comment(parent_comment_id)

                        parent_timestamp = datetime.fromtimestamp(parent.created_utc)
                        comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)

                        comment_array.append(this_comment.body)
                        comment_toxicities.append(sentiment_toxicity(this_comment.body, **tokenizer_kwargs)[0]['score'])

                        timestamps.append(comment_timestamp)
                        reply_timestamps.append((parent_timestamp, comment_timestamp))

                except Exception as e:
                    print(e)
                    continue

                for i in range(len(comment_array)):
#                         comment_array[i] = normalize(nlp((comment_array[i]))) # normalizes comment but leaves as string
                    comment_array[i] = nlp(normalize(nlp((comment_array[i])))) # this normalizes comment and wraps in nlp

                comment_similarities = []
                time_intervals = []
                response_intervals = []

                for sent_1, sent_2 in combinations(comment_array, 2):

#                         comment_similarities.append(calculate_cosine_sim(sent_1, sent_2)) checks cosine simularity of each comment against the next 
                    comment_similarities.append(sent_1.similarity(sent_2)) # nlp similiarity

                successive_times = list(pairwise(timestamps))
                for pair in successive_times: # calculates the intervals between user's comments
                    time_intervals.append(abs(pair[0] - pair[1]))

                for pair in reply_timestamps:
                    response_intervals.append(abs(pair[1] - pair[0]))# calculates how quickly a comment replied to its parent

                try:
                    time_data = pd.Series(time_intervals)
                    avg_time_diff = (time_data.sum()/len(time_data)).round('1s')
                except Exception as e:
                    print(e)
                    continue

                try:
                    response_data = pd.Series(response_intervals)
                    avg_reply_speed = (response_data.sum()/len(response_data)).round('1s')
                except Exception as e:
                    print(e)
                    continue

                try:
                    avg_comment_similarity = sum(comment_similarities)/len(comment_similarities)
                    avg_toxicity = sum(comment_toxicities)/len(comment_toxicities)
                except ZeroDivisionError:
                    continue
                except Exception as e:
                    print(e)
                    continue
                try:
                    user_dict['comment_karma'] = comment_karma
                    user_dict['submission_karma'] = submission_karma
                    user_dict['karma_ratio'] = karma_ratio
                    user_dict['avg_comment_similarity'] = avg_comment_similarity # formating dict
                    user_dict['avg_toxicity'] = avg_toxicity
                    user_dict['avg_reply_speed'] = avg_reply_speed
                    user_dict['avg_comment_time_interval'] = avg_time_diff
#                         user_dict['comment_intervals'] = time_intervals

                    rows.append(user_dict) 
                except Exeception as e:
                    print(e)
                    continue

    except Exception as e:
        print(e)
        continue

users_df = pd.DataFrame.from_dict(rows, orient='columns')
users_df.to_csv(f'{path}/{user_dir}/raw_known_bots.csv')

### Streaming Feature Extraction

In [ ]:
rows = []

for submission in botdefense.stream.submissions():
    try:
        user_name = submission.title[13:]

        user_dict = {}
        user_dict['username'] = user_name
        user = reddit.redditor(user_name)

        comment_karma = user.comment_karma
        submission_karma = user.link_karma

        karma_ratio = round(comment_karma/submission_karma, 4)

        comment_array = []
        timestamps = []
        reply_timestamps = []
        comment_toxicities = []


        for this_comment in user.comments.new(limit=15):

            parent_comment_id = this_comment.parent_id
            if parent_comment_id.startswith('t3'):
                parent_comment_id = parent_comment_id[3:]
                parent = reddit.submission(parent_comment_id)
            else:
                parent = reddit.comment(parent_comment_id)

            parent_timestamp = datetime.fromtimestamp(parent.created_utc)
            comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)

            comment_array.append(this_comment.body)
            comment_toxicities.append(sentiment_toxicity(this_comment.body, **tokenizer_kwargs)[0]['score'])

            timestamps.append(comment_timestamp)
            reply_timestamps.append((parent_timestamp, comment_timestamp))

        for i in range(len(comment_array)):
            comment_array[i] = nlp(normalize(nlp((comment_array[i]))))

        comment_similarities = []
        time_intervals = []
        response_intervals = []

        for sent_1, sent_2 in combinations(comment_array, 2):
            comment_similarities.append(sent_1.similarity(sent_2))

        successive_times = list(pairwise(timestamps))
        for pair in successive_times: # calculates the intervals between user's comments
            time_intervals.append(abs(pair[0] - pair[1]))
        for pair in reply_timestamps:
            response_intervals.append(abs(pair[1] - pair[0]))

        time_data = pd.Series(time_intervals)
        avg_time_diff = (time_data.sum()/len(time_data)).round('1s')

        response_data = pd.Series(response_intervals)
        avg_reply_speed = (response_data.sum()/len(response_data)).round('1s')

        avg_comment_similarity = sum(comment_similarities)/len(comment_similarities)
        avg_toxicity = sum(comment_toxicities)/len(comment_toxicities)

        user_dict['comment_karma'] = comment_karma
        user_dict['submission_karma'] = submission_karma
        user_dict['karma_ratio'] = karma_ratio
        user_dict['avg_comment_similarity'] = avg_comment_similarity # formating dict
        user_dict['avg_toxicity'] = avg_toxicity
        user_dict['avg_reply_speed'] = avg_reply_speed
        user_dict['avg_comment_time_interval'] = avg_time_diff

        print(user_dict)
        rows.append(user_dict) 

    except Exception as e:
        print(e)

users_df = pd.DataFrame.from_dict(rows, orient='columns')
users_df.to_csv(f'{path}/{user_dir}/suspected_bots_2.csv')

### For suspected bots

In [15]:
# botdefense = reddit.subreddit('botdefense').new(limit=500)
# user_names = []

# for submission in botdefense:
#     user_names.append(submission.title[13:])
# user_names = user_names[1:]
# print(user_names)

['Apartexalt698', 'AgePsycholog', 'South_Tensio', 'Independentfdy', 'SenorBeef', 'Aggressive_Finger579', 'Downtowarkb', 'xansochi', 'Backgroundbv', 'MixedRapidity', 'mynkamerve', 'legoholamoo', 'cuspiafc', 'kakashi4ok', 'thierryleblanyi', 'Fhaingsk', 'Barbecx', 'Lakapparygd', 'UpstateAlight72', 'chitatsjy', 'conkeysoundhq', 'FickleEntrepreneu', 'Automatic_Suc', 'Unhapolasvgf', 'kubafundipd', 'fondsbrilhz', 'IvaSwinton', 'CryptoColon', 'Mysterious_Fer', 'HistoricalAm', 'No-Education-4082', 'ElliotDictator', 'Individual_Drink162', 'illitsPoordrr', 'tominogrt', 'usadilozz', 'bakezillapk', 'vossejarhe', 'tropotbw', 'mioasislq', 'dapaulasafh', 'jarbythedoorkh', 'acicarronb', 'vatinukasgh', 'cosmicdancersjk', 'e2r3a3kukys', '1glasib', 'w8229549', 'outrightfoal480', 'Horror_Constan', 'Typicalecovergd', 'CommercialClient51', 'moidwayinfonixc', 'pc29lo', 'ntyriminger', 'initialshuffler', 'Suitable-Method3245', 'NulkStomomocg', 'Subartaspz', 'Acceptable_Roun', 'EmbarrassedCoun', 'Andresallison99

In [12]:
rows = []
for user_name in tqdm(user_names):
    try:
        user_dict = {}
        user = reddit.redditor(user_name)
        user_dict['username'] = user_name
        user = reddit.redditor(user_name)

        if get_user_status(user) == 'active' and str(user_name) not in 'nan': # checks that user isn't suspended/deleted
            if not user.is_mod: #ignore mods

                comment_array = []
                timestamps = []
                reply_timestamps = []
                comment_toxicities = []

                comment_karma = user.comment_karma
                submission_karma = user.link_karma

                karma_ratio = round(comment_karma/submission_karma, 4)

                try:
                    for this_comment in user.comments.new(limit=15):

                        parent_comment_id = this_comment.parent_id
                        if parent_comment_id.startswith('t3'):
                            parent_comment_id = parent_comment_id[3:]
                            parent = reddit.submission(parent_comment_id)
                        else:
                            parent = reddit.comment(parent_comment_id)

                        parent_timestamp = datetime.fromtimestamp(parent.created_utc)
                        comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)

                        comment_array.append(this_comment.body)
                        comment_toxicities.append(sentiment_toxicity(this_comment.body, **tokenizer_kwargs)[0]['score'])

                        timestamps.append(comment_timestamp)
                        reply_timestamps.append((parent_timestamp, comment_timestamp))

                except Exception as e:
                    print(e)
                    continue

                for i in range(len(comment_array)):
#                         comment_array[i] = normalize(nlp((comment_array[i]))) # normalizes comment but leaves as string
                    comment_array[i] = nlp(normalize(nlp((comment_array[i])))) # this normalizes comment and wraps in nlp

                comment_similarities = []
                time_intervals = []
                response_intervals = []

                for sent_1, sent_2 in combinations(comment_array, 2):

#                         comment_similarities.append(calculate_cosine_sim(sent_1, sent_2)) checks cosine simularity of each comment against the next 
                    comment_similarities.append(sent_1.similarity(sent_2)) # nlp similiarity

                successive_times = list(pairwise(timestamps))
                for pair in successive_times: # calculates the intervals between user's comments
                    time_intervals.append(abs(pair[0] - pair[1]))

                for pair in reply_timestamps:
                    response_intervals.append(abs(pair[1] - pair[0]))# calculates how quickly a comment replied to its parent

                try:
                    time_data = pd.Series(time_intervals)
                    avg_time_diff = (time_data.sum()/len(time_data)).round('1s')
                except Exception as e:
                    print(e)
                    continue

                try:
                    response_data = pd.Series(response_intervals)
                    avg_reply_speed = (response_data.sum()/len(response_data)).round('1s')
                except Exception as e:
                    print(e)
                    continue

                try:
                    avg_comment_similarity = sum(comment_similarities)/len(comment_similarities)
                    avg_toxicity = sum(comment_toxicities)/len(comment_toxicities)
                except ZeroDivisionError:
                    continue
                except Exception as e:
                    print(e)
                    continue
                try:
                    user_dict['comment_karma'] = comment_karma
                    user_dict['submission_karma'] = submission_karma
                    user_dict['karma_ratio'] = karma_ratio
                    user_dict['avg_comment_similarity'] = avg_comment_similarity # formating dict
                    user_dict['avg_toxicity'] = avg_toxicity
                    user_dict['avg_reply_speed'] = avg_reply_speed
                    user_dict['avg_comment_time_interval'] = avg_time_diff
#                         user_dict['comment_intervals'] = time_intervals

                    rows.append(user_dict) 
                except Exeception as e:
                    print(e)
                    continue

    except Exception as e:
        print(e)
        continue

users_df = pd.DataFrame.from_dict(rows, orient='columns')
users_df.to_csv(f'{path}/{user_dir}/known_bots.csv')

  0%|                                                                               | 3/2500 [00:06<1:33:11,  2.24s/it]C:\Users\mhous\AppData\Local\Temp\ipykernel_6568\4106614434.py:66: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

C:\Users\mhous\AppData\Local\Temp\ipykernel_6568\4106614434.py:67: RuntimeWarning:

invalid value encountered in double_scalars

  0%|▏                                                                              | 4/2500 [00:07<1:20:33,  1.94s/it]

'str' object cannot be interpreted as an integer


  1%|▉                                                                               | 31/2500 [01:22<58:27,  1.42s/it]

sus


  2%|█▎                                                                           | 43/2500 [04:52<18:50:28, 27.61s/it]

...

100%|██████████████████████████████████████████████████████████████████████████▋| 2491/2500 [11:42:56<02:15, 15.06s/it]

sus


100%|██████████████████████████████████████████████████████████████████████████▊| 2492/2500 [11:43:07<01:49, 13.64s/it]

sus


100%|███████████████████████████████████████████████████████████████████████████| 2500/2500 [11:48:18<00:00, 17.00s/it]


### Streaming Feature Extraction

In [ ]:
rows = []

for submission in botdefense.stream.submissions():
    try:
        user_name = submission.title[13:]

        user_dict = {}
        user_dict['username'] = user_name
        user = reddit.redditor(user_name)

        comment_karma = user.comment_karma
        submission_karma = user.link_karma

        karma_ratio = round(comment_karma/submission_karma, 4)

        comment_array = []
        timestamps = []
        reply_timestamps = []
        comment_toxicities = []


        for this_comment in user.comments.new(limit=15):

            parent_comment_id = this_comment.parent_id
            if parent_comment_id.startswith('t3'):
                parent_comment_id = parent_comment_id[3:]
                parent = reddit.submission(parent_comment_id)
            else:
                parent = reddit.comment(parent_comment_id)

            parent_timestamp = datetime.fromtimestamp(parent.created_utc)
            comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)

            comment_array.append(this_comment.body)
            comment_toxicities.append(sentiment_toxicity(this_comment.body, **tokenizer_kwargs)[0]['score'])

            timestamps.append(comment_timestamp)
            reply_timestamps.append((parent_timestamp, comment_timestamp))

        for i in range(len(comment_array)):
            comment_array[i] = nlp(normalize(nlp((comment_array[i]))))

        comment_similarities = []
        time_intervals = []
        response_intervals = []

        for sent_1, sent_2 in combinations(comment_array, 2):
            comment_similarities.append(sent_1.similarity(sent_2))

        successive_times = list(pairwise(timestamps))
        for pair in successive_times: # calculates the intervals between user's comments
            time_intervals.append(abs(pair[0] - pair[1]))
        for pair in reply_timestamps:
            response_intervals.append(abs(pair[1] - pair[0]))

        time_data = pd.Series(time_intervals)
        avg_time_diff = (time_data.sum()/len(time_data)).round('1s')

        response_data = pd.Series(response_intervals)
        avg_reply_speed = (response_data.sum()/len(response_data)).round('1s')

        avg_comment_similarity = sum(comment_similarities)/len(comment_similarities)
        avg_toxicity = sum(comment_toxicities)/len(comment_toxicities)

        user_dict['comment_karma'] = comment_karma
        user_dict['submission_karma'] = submission_karma
        user_dict['karma_ratio'] = karma_ratio
        user_dict['avg_comment_similarity'] = avg_comment_similarity # formating dict
        user_dict['avg_toxicity'] = avg_toxicity
        user_dict['avg_reply_speed'] = avg_reply_speed
        user_dict['avg_comment_time_interval'] = avg_time_diff

        print(user_dict)
        rows.append(user_dict) 

    except Exception as e:
        print(e)

users_df = pd.DataFrame.from_dict(rows, orient='columns')
users_df.to_csv(f'{path}/{user_dir}/suspected_bots_2.csv')